In [1]:
import googlemaps
gmaps_key=input()
gmaps = googlemaps.Client(key=gmaps_key)

## 좌표와 주소가 필요함.

In [2]:
tmp=gmaps.geocode('서울영등포경찰서', language='ko')

In [9]:
tmp[0].get('geometry')['location']          # 좌표
lat = tmp[0].get('geometry')['location']['lat']
lng = tmp[0].get('geometry')['location']['lng']

print(lat, lng)

37.5260441 126.9008091


In [8]:
tmp[0].get('formatted_address')             # 주소

'대한민국 서울특별시 영등포구 국회대로 608'

In [11]:
tmp[0].get('formatted_address').split()[2]

'영등포구'

---
# 데이터 가져오기

In [12]:
import pandas as pd
import numpy as np

crime_raw_data = pd.read_csv('../data2/02. crime_in_Seoul.csv', thousands=',', encoding='euc-kr')
crime_raw_data = crime_raw_data[crime_raw_data['죄종'].notnull()]
crime_station = crime_raw_data.pivot_table(
    crime_raw_data, 
    index='구분', 
    columns=['죄종','발생검거'], 
    aggfunc=[np.sum]
)
crime_station.columns=crime_station.columns.droplevel([0,1])
crime_station.head()

죄종       강간           강도          살인           절도              폭력        
발생검거     검거     발생    검거    발생    검거   발생      검거      발생      검거      발생
구분                                                                       
강남    269.0  339.0  26.0  24.0   3.0  3.0  1129.0  2438.0  2096.0  2336.0
강동    152.0  160.0  13.0  14.0   5.0  4.0   902.0  1754.0  2201.0  2530.0
강북    159.0  217.0   4.0   5.0   6.0  7.0   672.0  1222.0  2482.0  2778.0
강서    239.0  275.0  10.0  10.0  10.0  9.0  1070.0  1952.0  2768.0  3204.0
관악    264.0  322.0  10.0  12.0   7.0  6.0   937.0  2103.0  2707.0  3235.0

---
# crime_station 에 새로운 컬럼 데이터 넣기

In [15]:
crime_station['구별'] = np.nan
crime_station['lat'] = np.nan
crime_station['lng'] = np.nan

In [16]:
crime_station.head()

죄종       강간           강도          살인           절도              폭력          구별  \
발생검거     검거     발생    검거    발생    검거   발생      검거      발생      검거      발생       
구분                                                                              
강남    269.0  339.0  26.0  24.0   3.0  3.0  1129.0  2438.0  2096.0  2336.0 NaN   
강동    152.0  160.0  13.0  14.0   5.0  4.0   902.0  1754.0  2201.0  2530.0 NaN   
강북    159.0  217.0   4.0   5.0   6.0  7.0   672.0  1222.0  2482.0  2778.0 NaN   
강서    239.0  275.0  10.0  10.0  10.0  9.0  1070.0  1952.0  2768.0  3204.0 NaN   
관악    264.0  322.0  10.0  12.0   7.0  6.0   937.0  2103.0  2707.0  3235.0 NaN   

죄종   lat lng  
발생검거          
구분            
강남   NaN NaN  
강동   NaN NaN  
강북   NaN NaN  
강서   NaN NaN  
관악   NaN NaN

- 경찰서 이름 위에서 소속된 구 이름 얻기
- 구 이름과 위도 경도 정보를 저장할 준비
- 반복문을 이용해서 위 표의 NaN을 모두 채워주기
- iterrows() 함수 사용하기

In [19]:
count=0

for idx, rows in crime_station.iterrows():
    # print(idx)
    station_name = '서울' + str(idx) + '경찰서'
    # print(station_name)
    tmp = gmaps.geocode(station_name, language='ko')

    tmp_gu = tmp[0].get('formatted_address')
    lat = tmp[0].get('geometry')['location']['lat']
    lng = tmp[0].get('geometry')['location']['lng']

    crime_station.loc[idx, 'lat'] = lat
    crime_station.loc[idx, 'lng'] = lng
    crime_station.loc[idx, '구별'] = tmp_gu.split()[2]

    print(count)
    count = count  + 1

서울강남경찰서
서울강동경찰서
서울강북경찰서
서울강서경찰서
서울관악경찰서
서울광진경찰서
서울구로경찰서
서울금천경찰서
서울남대문경찰서
서울노원경찰서
서울도봉경찰서
서울동대문경찰서
서울동작경찰서
서울마포경찰서
서울방배경찰서
서울서대문경찰서
서울서부경찰서
서울서초경찰서
서울성동경찰서
서울성북경찰서
서울송파경찰서
서울수서경찰서
서울양천경찰서
서울영등포경찰서
서울용산경찰서
서울은평경찰서
서울종로경찰서
서울종암경찰서
서울중랑경찰서
서울중부경찰서
서울혜화경찰서


In [20]:
crime_station.head()

죄종       강간           강도          살인           절도              폭력          구별  \
발생검거     검거     발생    검거    발생    검거   발생      검거      발생      검거      발생       
구분                                                                              
강남    269.0  339.0  26.0  24.0   3.0  3.0  1129.0  2438.0  2096.0  2336.0 NaN   
강동    152.0  160.0  13.0  14.0   5.0  4.0   902.0  1754.0  2201.0  2530.0 NaN   
강북    159.0  217.0   4.0   5.0   6.0  7.0   672.0  1222.0  2482.0  2778.0 NaN   
강서    239.0  275.0  10.0  10.0  10.0  9.0  1070.0  1952.0  2768.0  3204.0 NaN   
관악    264.0  322.0  10.0  12.0   7.0  6.0   937.0  2103.0  2707.0  3235.0 NaN   

죄종   lat lng  
발생검거          
구분            
강남   NaN NaN  
강동   NaN NaN  
강북   NaN NaN  
강서   NaN NaN  
관악   NaN NaN

---
# df 의 column 정리하기

In [24]:
tmp = [
    crime_station.columns.get_level_values(0)[n] + crime_station.columns.get_level_values(1)[n]
    for n in range(0, len(crime_station.columns.get_level_values(0)))
]       # list comprehence
tmp

['강간검거',
 '강간발생',
 '강도검거',
 '강도발생',
 '살인검거',
 '살인발생',
 '절도검거',
 '절도발생',
 '폭력검거',
 '폭력발생',
 '구별',
 'lat',
 'lng']

In [25]:
tmp, len(tmp), len(crime_station.columns.get_level_values(0))

(['강간검거',
  '강간발생',
  '강도검거',
  '강도발생',
  '살인검거',
  '살인발생',
  '절도검거',
  '절도발생',
  '폭력검거',
  '폭력발생',
  '구별',
  'lat',
  'lng'],
 13,
 13)

In [26]:
crime_station.columns = tmp

In [27]:
crime_station.head()

,강간검거,강간발생,강도검거,강도발생,살인검거,살인발생,절도검거,절도발생,폭력검거,폭력발생,구별,lat,lng
구분,,,,,,,,,,,,,
강남,269.0,339.0,26.0,24.0,3.0,3.0,1129.0,2438.0,2096.0,2336.0,NaN,NaN,NaN
강동,152.0,160.0,13.0,14.0,5.0,4.0,902.0,1754.0,2201.0,2530.0,NaN,NaN,NaN
강북,159.0,217.0,4.0,5.0,6.0,7.0,672.0,1222.0,2482.0,2778.0,NaN,NaN,NaN
강서,239.0,275.0,10.0,10.0,10.0,9.0,1070.0,1952.0,2768.0,3204.0,NaN,NaN,NaN
관악,264.0,322.0,10.0,12.0,7.0,6.0,937.0,2103.0,2707.0,3235.0,NaN,NaN,NaN


In [28]:
# 데이터 저장
# crime_station.to_csv('../data2/02. crime_in_Seoul_raw.csv', sep=',', encoding='utf-8')

In [30]:
pd.read_csv('../data2/02. crime_in_Seoul_raw.csv')


,구분,강간검거,강간발생,강도검거,강도발생,살인검거,살인발생,절도검거,절도발생,폭력검거,폭력발생,구별,lat,lng
0,강남,269.0,339.0,26.0,24.0,3.0,3.0,1129.0,2438.0,2096.0,2336.0,강남구,37.509435,127.066958
1,강동,152.0,160.0,13.0,14.0,5.0,4.0,902.0,1754.0,2201.0,2530.0,강동구,37.528511,127.126822
2,강북,159.0,217.0,4.0,5.0,6.0,7.0,672.0,1222.0,2482.0,2778.0,강북구,37.637388,127.027324
3,강서,239.0,275.0,10.0,10.0,10.0,9.0,1070.0,1952.0,2768.0,3204.0,양천구,37.539783,126.829997
4,관악,264.0,322.0,10.0,12.0,7.0,6.0,937.0,2103.0,2707.0,3235.0,관악구,37.474379,126.950975
5,광진,234.0,279.0,6.0,11.0,4.0,4.0,1057.0,2636.0,2011.0,2392.0,광진구,37.542873,127.083821
6,구로,181.0,273.0,13.0,10.0,9.0,9.0,861.0,1910.0,2680.0,3164.0,구로구,37.494931,126.886731
7,금천,143.0,175.0,7.0,7.0,6.0,6.0,654.0,1264.0,1946.0,2193.0,관악구,37.481405,126.909951
8,남대문,52.0,57.0,4.0,5.0,1.0,1.0,429.0,946.0,832.0,890.0,중구,37.554758,126.973498
9,노원,142.0,159.0,9.0,6.0,6.0,5.0,740.0,1857.0,2124.0,2516.0,노원구,37.642361,127.071403
